# Рисуем интерактивные карты с Folium

В питоне есть множество библиотек, с помощью которых можно рисовать и анализировать пространственную информацию (spatial analysis).

Вот некоторые из них:

* folium
* gmaps
* basemap
* cartopy
* geoplotlib


В этом туториале пойдет речь о библиотеке Folium (https://github.com/python-visualization/folium), которая представляет собой питон-обертку над JS библиотекой Leaflet.

"Manipulate your data in Python, then visualize it in on a Leaflet map via Folium." (c) github

Большим плюсом по сравнению с другими библиотеками является интерактивность. Карты можно зумить, исследовать, кликать на маркеры, создать сложные типы визуализации.
Минусом является производительность. Создание карты с большим количеством точек может составлять минуты.

К сожалению, информация по работе с библиотекой раскидана по разным сайтам и туториалам. В официальной документации информации очень мало, а в русскоязычном интернете вообще ничего нет.

Будем исследовать датасет с пабами Москвы =). 

Датасет в виде списка пабов Москвы с их координатами выгружен с http://openstreetmap.ru
В конце туториала также попробуем отобразить кратчайший путь через пабы Москвы, идея чего нагло украдена отсюда: http://www.math.uwaterloo.ca/tsp/pubs/

## Подготовка данных

In [252]:
import json
import pandas as pd
import folium
import requests

with open('pubs.json') as json_data:
    d = json.load(json_data)
    
columns = ['lat', 'lon', 'name_ru', 'opening_hours', 'website']
index = range(0, len(d["data"]))
pubs = pd.DataFrame(columns = columns, index = index)

for i in range(0, len(d["data"])):
    pubs['lat'][i] = d["data"][i]["lat"]
    pubs['lon'][i] = d["data"][i]["lon"]
    pubs['opening_hours'].iloc[i] = d["data"][i]["opening_hours"]
    pubs['website'][i] = d["data"][i]["website"]
    pubs['name_ru'][i] = d["data"][i]["name_ru"]    

In [253]:
pubs.head(3)

,lat,lon,name_ru,opening_hours,website
0,55.6051,37.3538,Московский дворик,None,None
1,55.9092,37.8661,Кружка,None,None
2,55.8649,37.6056,Кружка,None,http://www.kruzhka.ru/


In [254]:
# Для центрирования карт я выбрала центральную точку Москвы

kremlin = [55.750730, 37.617322]

## Dot density map

Самый простой вариант анализа - отобразить данные как точки (или маркеры) на карте. В popup (выноску) положим название заведения и часы работы.

Карта инициализируется с помощью синтаксиса `map = folium.Map(location=kremlin, zoom_start=11)`.

Добавляем маркеры `folium.Marker().add_to(map)`.

Различные типы маркеров задаются функциями:

- Marker (использую для визуализации ниже)
- RegularPolygonMarker
- CircleMarker
- PolygonMarker

Возможные атрибуты: 

- color
- fill_color
- weight
- radius
- number_of_sides (для RegularPolygonMarker)

Так же в выноску можно передавать график vincent (https://github.com/wrobstory/vincent) с помощью синтаксиса: 

`folium.Popup().add_child(folium.Vega())`

`tiles` - источник тайлов карт. Я обычно использую openstreetmaps - они идут по умолчанию.

In [255]:
pubs_map = folium.Map(location=kremlin, zoom_start=11)

for i in range(0, len(pubs)):
    folium.Marker([pubs['lat'][i], pubs['lon'][i]], popup = str(pubs['name_ru'][i]) + ": " 
                  + str(pubs['opening_hours'][i])).add_to(pubs_map)
    
pubs_map

Cluster marker - раскраска в зависимости от плотности точек. Близкие точки сливаются в один маркер.

Судя по данным, наибольшая плотность пабов в Москве - на серево-востоке от Кремля, в районе Чистых прудов.

In [256]:
from folium import features

pubs_map = folium.Map(location=kremlin, zoom_start=12)
mc = features.MarkerCluster()

for i in range(0, len(pubs)):
    mk = features.Marker([pubs['lat'][i], pubs['lon'][i]])
    mc.add_child(mk)
    
pubs_map.add_child(mc)

## Heatmap

Построим Heatmap распределения пабов по Москве

In [257]:
from folium import plugins
import numpy as np
import random

# с помощью рандома создаю оценки заведений. можете доработать датасет и найти реальные скопления хороших пабов.
pubs['star'] = [1 for i in range(len(pubs))]

pubs_map = folium.Map(location=kremlin, zoom_start=10)

data = [[x[0], x[1], x[2]] for x in np.array(pubs[['lat', 'lon', 'star']])]

HeatMap(data, radius = 20).add_to(pubs_map)
pubs_map

## Линии

Добавим на карту линии, соединяющую локации. Она создаётся с помощью folium.PolyLine, который принимает координаты соединяемых точек, и настраивается параметрами, схожими с параметрами маркеров.

Здесь мы выбрали и соединили 4 заведения в Сокольниках:

In [259]:
sololniki = [55.791981, 37.664456]

pubs_map_sokolniki = folium.Map(location=sololniki, zoom_start=13)
path  = []
for i in [100, 101, 102, 103]:
    folium.Marker([pubs['lat'][i], pubs['lon'][i]], popup = str(pubs['name_ru'][i]) + ": " 
                  + str(pubs['opening_hours'][i])).add_to(pubs_map_sokolniki)
    path.append([[pubs['lat'][i], pubs['lon'][i]], [pubs['lat'][i+1], pubs['lon'][i+1]]])

folium.PolyLine(path[0:3], color='blue', weight=4, opacity=0.7, popup=str(i)).add_to(pubs_map_sokolniki)
pubs_map_sokolniki   

## Найдем и отобразим кратчайший путь через пабы Москвы :)

Для уменьшения размера выборки выбираю только пабы с наивысшими оценками.

Для нахождения кратчайшего пути использую библиотеку Google or-tools, которая включает в себя алгоритмы решения задач нахождения маршрута. Работа с ней это тема для отдельного туториала, поэтому загружаю найденное решение из внешнего файла.
Пути между заведениями тут уже строятся не отрезками, соединяющими пары пабов, а следуют кратчайшему маршруту из паба в паб, подобранному с помощью http://project-osrm.org

In [260]:
pubs_final = pubs[pubs['star'] == 5]

pubs_final.to_csv('pubs.csv', sep = ';')

Have fun =)